# Matched wavelength photometry

**Version 0.1**

For today's problem, we will be perform matched-aperture photometry in 5 bands on multiple galaxies within a rich galaxy cluster. Ultimately, we will be looking for trends in galaxy colors and other properties as a function of cluster radius.

Note - we will use `astropy` for these tasks, though the use of [`Source Extractor`](https://www.astromatic.net/software/sextractor) is more standard within the galaxy community.


* * *

By M Alpaslan (Columbia) & AA Miller (CIERA/Northwestern & Adler)

## Problem 0) Install photutils

If you have not already done so, install the [`photutils`](http://photutils.readthedocs.io/en/stable/index.html) package from the `astropy` conda channel *within your DSFP environment*.

    conda install -c astropy photutils

In [1]:
import numpy as np
import pandas as pd
import astropy
from astropy.io import fits
from photutils.aperture import CircularAperture, CircularAnnulus, EllipticalAperture, EllipticalAnnulus
import matplotlib.pyplot as plt

%matplotlib notebook

## Problem 1) Download and Examine the Data

The images for this exercise can be downloaded from here: **URL needed**. Be sure to place the images in the same directory as this notebook (but do not add them to your git repo!).

**Problem 1a**

Display the $r$-band image. Use a logarithmic stretch.

In [ ]:
r_filename = ""
r_data = fits.getdata(r_filename)

plt.imshow(r_data, cmap='viridis', norm=LogNorm())

**Problem 1b**

Roughly how many sources are present in the image?

*Hint* - an exact count is not required here.

**Solution 1b**

*Write your answer here*